# Factor de Friccion con PySR

In [1]:
## Importando Librerias 
import numpy as np
import pandas as pd
from pysr import PySRRegressor
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


### Cargando datos

In [2]:
# cargando datos

# importacion de DF
file_path = 'C:\\Users\\lucas\\Documents\\VSCode\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_25.txt'
df_ff_25 = pd.read_csv(file_path, delimiter=',')

file_path = 'C:\\Users\\lucas\\Documents\\VSCode\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_53.txt'
df_ff_53 = pd.read_csv(file_path, delimiter=',')

file_path = 'C:\\Users\\lucas\\Documents\\VSCode\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_74.txt'
df_ff_74 = pd.read_csv(file_path, delimiter=',')

file_path = 'C:\\Users\\lucas\\Documents\\VSCode\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_102.txt'
df_ff_102 = pd.read_csv(file_path, delimiter=',')

In [3]:
# definiendo conjuntos de train y test

# definiendo conjunto de train
df_ff_train = pd.concat([df_ff_25, df_ff_74], ignore_index=True)

# separando entre x e y
y_train = df_ff_train.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_train = df_ff_train.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','frictionFactor'])

# definiendo conjunto de test

y_test = df_ff_53.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_test = df_ff_53.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','frictionFactor'])

# renombrando
X_train.rename(columns={'Rem':'Re'}, inplace=True)
X_test.rename(columns={'Rem':'Re'}, inplace=True)

### Entrenando Modelo

In [13]:
model = PySRRegressor(
    niterations=100,
    binary_operators=["+", "*", "pow"],
    unary_operators=[],
    population_size=200,
    verbosity=1,
    maxsize=12,
    parsimony=1e-3,
    progress=True
)

model.fit(X_train, y_train)

d:\env\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:73: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant or varia


Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 4.180e+01
Head worker occupation: 0.0%
Progress: 1 / 1500 total iterations (0.067%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
3           5.588e+00  5.314e+00  y = -2.4264 * -1.644
5           2.871e+00  3.331e-01  y = 2.5313 + (K ^ -1.4303)
7           9.306e-01  5.632e-01  y = (2.5313 * (K ^ -1.4303)) + 0.9818
9           8.920e-01  2.119e-02  y = (K ^ -1.1974) + (2.4334 * (K ^ -1.2183))
-------------------------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                           4.239675   
	1  >>>>  0.455932                        3.4347172 * (K ^ -1.209575)   
	2        0.001579          ((0.06762905 + K) ^ -1.339577) * 3.739836   
	3        0.136316  (7.810259 * (K ^ -1.2095693)) * (Re ^ -0.10515...   
	4        0.030612  ((5.919568 * ((Re ^ K) ^ -0.3994728)) * 4.9490...   
	
	       loss  complexity  
	0  5.525302           1  
	1  0.891910           5  
	2  0.889098           7  
	3  0.676935           9  
	4  0.636734          11  
]

In [15]:
best_equation = model.equations_.iloc[3]['equation']
print(best_equation)

(7.810259 * (K ^ -1.2095693)) * (Re ^ -0.10515685)


### Pruebas

In [9]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 0.8335812597548942
R^2: 0.8531704688664192


In [2]:
# comparar con modelo de Rafael

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
   return (7.810259 * (S ** -1.2095693)) * (Re ** -0.10515685)

In [8]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2}, R^2 cdrag GPLearn: {r2_gp} ")
print(f"MSE cdrag original: {mse}, MSE cdrag GPLearn: {mse_gp} ")
print(f"MAE cdrag original: {mae}, MAE cdrag GPLearn: {mae_gp} ")
print(f"MAPE cdrag original: {mape*100}*100, MAPE cdrag GPLearn: {mape_gp*100}*100 ")

R^2 cdrag original: 0.8700702078373823, R^2 cdrag GPLearn: 0.8603537943971947 
MSE cdrag original: 0.737637987361494, MSE cdrag GPLearn: 0.7928000524667923 
MAE cdrag original: 0.5184759754778028, MAE cdrag GPLearn: 0.500055257309874 
MAPE cdrag original: 11.262660298844454*100, MAPE cdrag GPLearn: 10.011761634618646*100 


In [4]:
model = PySRRegressor(
    niterations=100,
    binary_operators=["+", "*", "pow"],
    unary_operators=[],
    population_size=200,
    verbosity=1,
    maxsize=12,
    parsimony=1e-3,
    batching= True, 
    batch_size = 50,
    progress=True
)

model.fit(X_train, y_train)

d:\env\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Compiling Julia backend...


d:\env\lib\site-packages\pysr\sr.py:73: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant or variable (1). For more tips, please see https://astroautomata.com/PySR/tuning/
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 5.710e+02
Head worker occupation: 14.2%
Progress: 8 / 1500 total iterations (0.533%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.484e+01  1.594e+01  y = 1.1875
3           5.532e+00  4.934e-01  y = 3.7075 + 0.61197
5           1.298e+00  7.249e-01  y = (K ^ -2.2317) + 2.3113
7           9.035e-01  1.811e-01  y = ((0.366 * K) ^ -1.6201) ^ 0.76262
9           8.930e-01  5.814e-03  y = ((0.43081 * K) ^ -1.2869) + (0.47315 ^ K)
11          8.897e-01  1.865e-03  y = 1.9025 + (((0.76365 * (K ^ K)) * K) ^ -1.3744)
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 1.190e+03
Head worker occupation: 14.8%
Progress: 42 / 1500 total iterations (2.800%)
Hall of Fame:
---------------------------------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          4.2397304   
	1  >>>>  0.455918                       (K ^ -1.2069694) * 3.4424295   
	2        0.000577        (3.9172964 * (K ^ -1.0991151)) + -0.4865772   
	3        0.134310  (4.0058618 + (Re * -0.00017393178)) * (K ^ -1....   
	4        0.000013  (4.0058618 + ((Re + -0.9369057) * -0.000173931...   
	
	       loss  complexity  
	0  5.525302           1  
	1  0.891960           5  
	2  0.890931           7  
	3  0.681058           9  
	4  0.681040          11  
]

### Entrenamiento del Modelo V2

In [4]:
model = PySRRegressor(
    niterations=100,
    binary_operators=["+","-","*" ,"/", "^"],
    unary_operators=[],
    population_size=200,
    verbosity=1,
    maxsize=12,
    parsimony=1e-3,
    #constraints={'^':(-1,1)},
    progress=True
)

model.fit(X_train, y_train)

d:\env\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:73: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant or varia

Compiling Julia backend...


[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 3.230e+02
Head worker occupation: 0.0%
Progress: 1 / 1500 total iterations (0.067%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
3           1.315e+01  5.314e+00  y = 0.99457 / K
5           6.146e+00  3.803e-01  y = (K / 0.28971) / K
7           1.047e+00  8.852e-01  y = ((K / 0.26726) / K) / K
9           8.910e-01  8.042e-02  y = (((K ^ -0.18768) / 0.28295) / K) - 0.1039
11          8.900e-01  5.822e-04  y = ((((K ^ 0.86

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          4.2395678   
	1        0.831940                                      3.7415504 / K   
	2        0.079924                        3.4347415 / (K ^ 1.2095841)   
	3  >>>>  0.126219                3.3642833 / ((-133.01392 / Re) + K)   
	4        0.080340            (3.012368 + ((1008.4439 / K) / Re)) / K   
	5        0.002740  (3.012368 + (((1008.4439 / K) / Re) ^ 0.927412...   
	
	       loss  complexity  
	0  5.525302           1  
	1  1.046507           3  
	2  0.891910           5  
	3  0.692929           7  
	4  0.590074           9  
	5  0.586850          11  
]

In [5]:
best_equation = model.equations_.iloc[3]['equation']
print(best_equation)

3.3642833 / ((-133.01392 / Re) + K)


In [6]:
best_equation = model.equations_.iloc[4]['equation']
print(best_equation)

(3.012368 + ((1008.4439 / K) / Re)) / K


In [7]:
best_equation = model.equations_.iloc[5]['equation']
print(best_equation)

(3.012368 + (((1008.4439 / K) / Re) ^ 0.92741257)) / K


In [8]:
# comparar con modelo de Rafael

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return 3.3642833 / ((-133.01392 / Re) + S)

In [9]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8556412300187617 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.8195542436769815 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.5838494169448822 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 13.253232740525625 % 


In [10]:
# otra ecuacion

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return (3.012368 + (((1008.4439 / S) / Re) ** 0.92741257)) / S

In [11]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8725221969866254 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.7237175437821176 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.5070969296702303 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 10.65400381526513 % 


In [12]:
# otra otra ecuacion

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return (3.012368 + ((1008.4439 / S) / Re)) / S


In [13]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.872511952120607 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.7237757059805234 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.5027946203357261 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 10.42229357196006 % 


### Experimento: RMSLE como funcion loss

In [4]:

rmsle_loss = """ function rmsle(prediction, target)
    # Calcular los logaritmos con un mínimo de 0.001 para evitar valores negativos o cero
    log_true = log.(max.(target, 0.001) .+ 1)
    log_pred = log.(max.(prediction, 0.001) .+ 1)

    # Calcular el error cuadrático medio logarítmico
    rmsle_value = (log_pred .- log_true).^2

    # Calcular el valor de RMSLE como la raíz cuadrada de la suma entre la cantidad de elementos
    return sqrt(sum(rmsle_value) / length(target))
end
"""

In [5]:
model = PySRRegressor(
    niterations=50,
    binary_operators=["+","-","*" ,"/", "^"],
    unary_operators=[],
    elementwise_loss= rmsle_loss,
    population_size=200,
    verbosity=1,
    maxsize=15,
    parsimony=1e-3,
    constraints={'^':(2,1)},
    nested_constraints={'^':{'^':0}},
    complexity_of_variables = 2,
    random_state = 0,
    progress=True
)

model.fit(X_train, y_train)

c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:1310: UserWarning: Note: Setting `random_state` without also setting `deterministic` to True and `procs` to 0 will result in non-deterministic searches. 
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(


Compiling Julia backend...


[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 750 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 2.870e+03
Head worker occupation: NaN%
Progress: 2 / 750 total iterations (0.267%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
3           6.946e-01  5.314e+00  y = 1.3646 - -0.055472
4           4.270e-01  4.865e-01  y = K + 1.2428
5           3.018e-01  3.469e-01  y = (0.83862 ^ -1.0472) + 1.5415
6           2.271e-01  2.843e-01  y = (1.5415 - -1.0472) / K
8           1.015e-01  4.029e-01  y = (1.0173 + (1.5415 - -0.81423

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          2.9841962   
	1  >>>>  0.353261                                      3.3706987 / K   
	2        0.074212                         3.388214 / (K ^ 1.2384768)   
	3        0.000516        (2.9407501 / (K ^ 1.3782816)) - -0.41557965   
	4        0.000992                  ((5.1570683 / K) - 2.8004696) + K   
	5        0.000159  (2.6069255 / ((K ^ 1.6616609) - -0.04202245)) ...   
	6        0.109519  (4.285438 / K) - (0.050914194 / (Re ^ -0.34750...   
	7        0.010988  (-0.38715765 - ((Re ^ -0.086198606) * ((-0.730...   
	
	       loss  complexity  
	0  0.292841           1  
	1  0.101479           4  
	2  0.087481           6  
	3  0.087391           8  
	4  0.087304           9  
	5  0.087290          10  
	6  0.078235          11  
	7  0.074871          15  
]

In [6]:
best_equation = model.equations_.iloc[5]['equation']
print(best_equation)

(2.6069255 / ((K ^ 1.6616609) - -0.04202245)) + 0.83342224


In [7]:
best_equation = model.equations_.iloc[6]['equation']
print(best_equation)

(4.285438 / K) - (0.050914194 / (Re ^ -0.34750924))


In [8]:
best_equation = model.equations_.iloc[7]['equation']
print(best_equation)

(-0.38715765 - ((Re ^ -0.086198606) * ((-0.7305493 / 0.08857655) / K))) - 0.3232977


In [9]:
# Comparando con modelo de Rafael

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return (-0.38715765 - ((Re ** (-0.086198606)) * ((-0.7305493 / 0.08857655) / S))) - 0.3232977

In [10]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8557477260703253 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.8189496438247857 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.5122415130588487 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 10.248581925481055 % 


In [13]:
# Comparando con modelo de Rafael con otra ecuacion 

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return (4.285438 / S) - (0.050914194 / (Re ** (-0.34750924)))

In [14]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8420548312988805 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.8966870062283204 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.5331336258641036 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 10.638404205270962 % 


### Entrenamiento con RMSLE como loss, pero sin tantas restricciones

In [ ]:

rmsle_loss = """ function rmsle(prediction, target)
    # Calcular los logaritmos con un mínimo de 0.001 para evitar valores negativos o cero
    log_true = log.(max.(target, 0.001) .+ 1)
    log_pred = log.(max.(prediction, 0.001) .+ 1)

    # Calcular el error cuadrático medio logarítmico
    rmsle_value = (log_pred .- log_true).^2

    # Calcular el valor de RMSLE como la raíz cuadrada de la suma entre la cantidad de elementos
    return sqrt(sum(rmsle_value) / length(target))
end
"""

In [6]:
model = PySRRegressor(
    niterations=30,
    binary_operators=["+","-","*" ,"/", "^"],
    unary_operators=[],
    elementwise_loss= rmsle_loss,
    population_size=200,
    verbosity=1,
    maxsize=15,
    parsimony=1e-3,
    constraints={'^':(-1,1)},
    #nested_constraints={'^':{'^':0}},
    complexity_of_variables = 2,
    random_state = 0,
    progress=True
)

model.fit(X_train, y_train)

c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:1310: UserWarning: Note: Setting `random_state` without also setting `deterministic` to True and `procs` to 0 will result in non-deterministic searches. 
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(


Compiling Julia backend...


[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 1.550e+03
Head worker occupation: 16.2%
Progress: 10 / 450 total iterations (2.222%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.520e+00  1.594e+01  y = 0.060488
3           2.939e-01  8.215e-01  y = 2.403 + 0.49259
5           2.929e-01  1.846e-03  y = (0.26759 ^ -0.83617) * 0.99392
6           8.807e-02  1.202e+00  y = (4.3909 / K) - 0.94902
8           8.803e-02  1.884e-04  y = (0.36182 ^ -1.0362) / (K + -0

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          2.9841404   
	1  >>>>  0.353261                                      3.3706908 / K   
	2        0.074212                        3.3882205 / (K ^ 1.2384909)   
	3        0.000516         (2.9340415 / (K ^ 1.3806713)) + 0.42177275   
	4        0.135406                3.0488415 / ((-230.97774 / Re) + K)   
	5        0.020283  ((Re ^ -0.12203351) * 9.012437) / (K ^ 1.2015067)   
	6        0.000082  ((9.012437 / (K ^ 1.2303771)) - -0.26413697) *...   
	7        0.002553  ((8.945306 * ((Re - -1.9277371) ^ -0.13399431)...   
	
	       loss  complexity  
	0  0.292841           1  
	1  0.101479           4  
	2  0.087481           6  
	3  0.087391           8  
	4  0.076324           9  
	5  0.073290          11  
	6  0.073278          13  
	7  0.072904          15  
]

In [7]:
best_equation = model.equations_.iloc[4]['equation']
print(best_equation)

3.0488415 / ((-230.97774 / Re) + K)


In [8]:
best_equation = model.equations_.iloc[5]['equation']
print(best_equation)

((Re ^ -0.12203351) * 9.012437) / (K ^ 1.2015067)


In [9]:
best_equation = model.equations_.iloc[6]['equation']
print(best_equation)

((9.012437 / (K ^ 1.2303771)) - -0.26413697) * (Re ^ -0.12587933)


In [10]:
best_equation = model.equations_.iloc[7]['equation']
print(best_equation)

((8.945306 * ((Re - -1.9277371) ^ -0.13399431)) / (K ^ 1.3017075)) - -0.3146223


In [11]:
# Ecuacion 7

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return ((8.945306 * ((Re +1.9277371) ** ( -0.13399431))) / (S ** (1.3017075))) + 0.3146223

In [12]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8629892566413002 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.7778379946265388 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.49759909599226576 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 10.138625668579326 % 


In [14]:
# Ecuacion 6

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return ((9.012437 / (S ** 1.2303771)) - -0.26413697) * (Re ** (-0.12587933))

# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8642805167380068 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.7705072471278994 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.4961014892821129 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 10.109137339829914 % 


In [16]:
# Ecuacion 5

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return ((Re ** (-0.12203351)) * 9.012437) / (S ** 1.2015067)

# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8640843547283954 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.7716208989513816 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.496466942828784 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 10.108521096511357 % 


### Experimento: repitiendo pero bajando parsimony (buscando aumentar simplicidad)

Resultados son peores al caso anterior

In [17]:
model = PySRRegressor(
    niterations=30,
    binary_operators=["+","-","*" ,"/", "^"],
    unary_operators=[],
    elementwise_loss= rmsle_loss,
    population_size=200,
    verbosity=1,
    maxsize=15,
    parsimony=1e-2,
    constraints={'^':(-1,1)},
    #nested_constraints={'^':{'^':0}},
    complexity_of_variables = 2,
    random_state = 0,
    progress=True
)

model.fit(X_train, y_train)

c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:1310: UserWarning: Note: Setting `random_state` without also setting `deterministic` to True and `procs` to 0 will result in non-deterministic searches. 
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
[ Info: Note: you are running with mo


Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
----------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          2.9841492   
	1  >>>>  0.353261                                      3.3706713 / K   
	2        0.074212                        3.3881848 / (K ^ 1.2384886)   
	3        0.000517         0.41087645 + (2.9459093 / (K ^ 1.3765196))   
	4        0.005448               (3.5606866 / K) - (6.507231e-5 * Re)   
	5        0.051655  ((4.439408 / K) - (Re ^ 0.118461296)) + 1.5983331   
	6        0.008815           ((4.988088 / K) - (Re ^ 0.12028212)) + K   
	7        0.000522  ((5.019278 / K) - ((Re * 1.7181648) ^ 0.113837...   
	8        0.010719  (2.990237 / (K ^ 1.3362371)) + (3.9821193 / ((...   
	
	       loss  complexity  
	0  0.292841           1  
	1  0.101479           4  
	2  0.087481           6  
	3  0.087391           8  
	4  0.086916           9  
	5  0.078385          11  
	6  0.077697          12  
	7  0.077616          14  
	8  0.076789          15  
]

In [18]:
best_equation = model.equations_.iloc[5]['equation']
print(best_equation)

((4.439408 / K) - (Re ^ 0.118461296)) + 1.5983331


In [19]:
best_equation = model.equations_.iloc[6]['equation']
print(best_equation)

((4.988088 / K) - (Re ^ 0.12028212)) + K


In [20]:
best_equation = model.equations_.iloc[7]['equation']
print(best_equation)

((5.019278 / K) - ((Re * 1.7181648) ^ 0.11383746)) + K


In [21]:
best_equation = model.equations_.iloc[8]['equation']
print(best_equation)

(2.990237 / (K ^ 1.3362371)) + (3.9821193 / ((0.062331628 * Re) * 0.062331613))


In [22]:
# Ecuacion 5

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return ((4.439408 / S) - (Re ** 0.118461296)) + 1.5983331

# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8445228980093445 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.8826752870476872 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.5255031252910931 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 10.560046221326628 % 


In [23]:
# Ecuacion 6

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return ((4.988088 / S) - (Re ** 0.12028212)) + S

# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8436237750087922 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.8877797920998295 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.5311911075750992 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 10.642646218305057 % 


In [25]:
# Ecuacion 7

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return (2.990237 / (S ** 1.3362371)) + (3.9821193 / ((0.062331628 * Re) * 0.062331613))

# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8441546335381335 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.8847659997228899 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.5289540258763367 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 10.648477351124292 % 


In [16]:
# Ecuacion 8

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return ((5.019278 / S) - ((Re * 1.7181648) ** 0.11383746)) + S

# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8433000632415955 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.8896175699679053 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.5312217334278918 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 10.664641191974516 % 


### Experimento: Probando con el mayor valor de parsimony

In [4]:
rmsle_loss = """ function rmsle(prediction, target)
    # Calcular los logaritmos con un mínimo de 0.001 para evitar valores negativos o cero
    log_true = log.(max.(target, 0.001) .+ 1)
    log_pred = log.(max.(prediction, 0.001) .+ 1)

    # Calcular el error cuadrático medio logarítmico
    rmsle_value = (log_pred .- log_true).^2

    # Calcular el valor de RMSLE como la raíz cuadrada de la suma entre la cantidad de elementos
    return sqrt(sum(rmsle_value) / length(target))
end
"""

In [17]:
model = PySRRegressor(
    niterations=30,
    binary_operators=["+","-","*" ,"/", "^"],
    unary_operators=[],
    elementwise_loss= rmsle_loss,
    population_size=200,
    verbosity=1,
    maxsize=20,
    parsimony=0.1,
    constraints={'^':(-1,1)},
    #nested_constraints={'^':{'^':0}},
    complexity_of_variables = 2,
    #procs=0, 
    #multithreading=False,
    #deterministic = True, 
    random_state = 0,
    progress=True
)

model.fit(X_train, y_train)

c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:1310: UserWarning: Note: Setting `random_state` without also setting `deterministic` to True and `procs` to 0 will result in non-deterministic searches. 
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
[ Info: Note: you are running with mo


Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 2.740e+04
Head worker occupation: 4.5%
Progress: 15 / 450 total iterations (3.333%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
3           2.928e-01  5.314e+00  y = 9.3201 / 3.1233
4           1.015e-01  1.060e+00  y = 3.3707 / K
6           8.748e-02  7.421e-02  y = 3.3882 / (K ^ 1.2385)
8           8.740e-02  4.461e-04  y = ((2.3457 / K) ^ 1.3256) + 0.27149
10          8.740e-02  4.351e-05  y = (((1.8436 - -0.26287) / K)

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                          2.9840946   
	1   >>>>  0.353261                                      3.3706722 / K   
	2         0.074212                        3.3882117 / (K ^ 1.2385334)   
	3         0.000516          (2.958702 / (K ^ 1.3719028)) + 0.39899606   
	4         0.001011                (4.6818733 / K) - (Re ^ 0.02452489)   
	5         0.011892   (Re ^ -0.15983897) + ((2.345473 / K) ^ 1.309099)   
	6         0.022548  (3.5455987 / (K ^ 1.1435984)) - ((0.0010368208...   
	7         0.057997  (4.800184 - (((0.009956512 * Re) * 0.009443051...   
	8         0.000053  ((4.800184 - K) - (((0.009956512 * Re) * 0.009...   
	9         0.084321  ((5.920717 ^ 4.014335) / ((Re * K) * K)) + (2....   
	10        0.000109  ((5.920717 ^ 4.014335) / (((Re * K) * K) - -1....   
	
	        loss  complexity  
	0   0.292841           1  
	1   0.101479           4  
	2   0.087481           6  
	3   0.087391           8  
	4   0.087303           9  
	5   0.085251          11  
	6   0.081492          13  
	7   0.076900          14  
	8   0.076892          16  
	9   0.070674          17  
	10  0.070659          19  
]

In [18]:
best_equation = model.equations_.iloc[4]['equation']
print(best_equation)

(4.6818733 / K) - (Re ^ 0.02452489)


In [28]:
best_equation = model.equations_.iloc[5]['equation']
print(best_equation)

(Re ^ -0.15983897) + ((2.345473 / K) ^ 1.309099)


In [25]:
best_equation = model.equations_.iloc[9]['equation']
print(best_equation)

((5.920717 ^ 4.014335) / ((Re * K) * K)) + (2.8723152 / K)


In [26]:
best_equation = model.equations_.iloc[9]['equation']
print(best_equation)

((5.920717 ^ 4.014335) / ((Re * K) * K)) + (2.8723152 / K)


In [27]:
best_equation = model.equations_.iloc[10]['equation']
print(best_equation)

((5.920717 ^ 4.014335) / (((Re * K) * K) - -1.2887985)) + (2.8723152 / K)


In [29]:
# Ecuacion 4

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return (4.6818733 / S) - (Re ** 0.02452489)

# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8287221895169936 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.9723791387755188 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.563163196063263 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 11.670189807443514 % 


In [30]:
# Ecuacion 5

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return (Re ** (-0.15983897)) + ((2.345473 / S) ** 1.309099)

# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8263355581403852 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.9859285329207186 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.568737522455391 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 11.32604107394076 % 


In [31]:
# Ecuacion 9

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
    return ((5.920717 ** 4.014335) / ((Re * S) * S)) + (2.8723152 / S)

# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Re']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Re']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 fFriction original: {r2}, R^2 fFriction GPLearn: {r2_gp} ")
print(f"MSE fFriction original: {mse}, MSE fFriction GPLearn: {mse_gp} ")
print(f"MAE fFriction original: {mae}, MAE fFriction GPLearn: {mae_gp} ")
print(f"MAPE fFriction original: {mape*100} %, fFriction cdrag GPLearn: {mape_gp*100} % ")

R^2 fFriction original: 0.8700702078373823, R^2 fFriction GPLearn: 0.8708308225978068 
MSE fFriction original: 0.737637987361494, MSE fFriction GPLearn: 0.7333198218992216 
MAE fFriction original: 0.5184759754778028, MAE fFriction GPLearn: 0.4841023236725421 
MAPE fFriction original: 11.262660298844454 %, fFriction cdrag GPLearn: 9.795863723383306 % 


### Experimento: incorporar exponentes de RDlS

In [4]:
model = PySRRegressor(
    niterations=30,
    binary_operators=["+","-","*" ,"/", "^"],
    unary_operators=["pot1(x)= x >= 0 ?  convert(typeof(x),x^(-1.1)) : convert(typeof(x), NaN)",
                    "pot2(x)=x >= 0 ?  convert(typeof(x),20*x^(-0.22)) : convert(typeof(x), NaN)"],
    extra_sympy_mappings={"pot1": lambda x:  x**(-1.1) , 
                        "pot2": lambda x: 20*x**(-0.22) },
    population_size=200,
    verbosity=1,
    maxsize=12,
    parsimony=1e-3,
    progress=True
)
model.fit(X_train, y_train)

d:\env\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


d:\env\lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(


Compiling Julia backend...


d:\env\lib\site-packages\pysr\sr.py:73: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant or variable (1). For more tips, please see https://astroautomata.com/PySR/tuning/
  warnings.warn(
[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 4.020e+02
Head worker occupation: 81.7%. This is high, and will prevent efficient resource usage. Increase `ncycles_per_iteration` to reduce load on head worker.
Progress: 4 / 450 total iterations (0.889%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
2           5.564e+00  7.971e+00  y = pot2(Re)
4           7.393e-01  1.009e+00  y = pot2(Re) / K
6           6.509e-01  6.364e-02  y = pot2(Re * K) / K
8           5.941e-01  4

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          3.7415972   
	1        0.036912                                           pot2(Re)   
	2        1.670867                                      3.7415972 / K   
	3  >>>>  0.347567                                       pot2(Re) / K   
	4        0.197387                                 pot1(K) * pot2(Re)   
	5        0.021166                         pot2(Re) / (K ^ 1.1397897)   
	6        0.003993         pot2(Re) / ((K ^ 1.2148508) + 0.022831168)   
	7        0.004905          pot1(K) * pot2(Re - pot2(K ^ 12.7634945))   
	8        0.001103  (pot1(K) * 0.9920696) * pot2(Re - pot2(K ^ 13....   
	
	       loss  complexity  
	0  5.773262           1  
	1  5.564044           2  
	2  1.046507           3  
	3  0.739257           4  
	4  0.606836           5  
	5  0.594127           6  
	6  0.589401           8  
	7  0.583647          10  
	8  0.582361          12  
]